# Sport Bets

#### Aqui você pegará todas as apostas do dia 10/10/2024 até 12/10/2024

Primeiro precisamos pegar os ids das apostas esportivas que foram feitas nesse periodo na tabela de transações de sport_book

In [ ]:
%%sql
select
    DISTINCT (cts.sport_bet_master_id)
from customer_transactions_sb cts
         JOIN customer_transaction_details ctd ON cts.customer_transaction_id = ctd.customer_transaction_id
         JOIN sport_bet_masters sbm on cts.sport_bet_master_id = sbm.sport_bet_master_id
         JOIN sport_selection ss on sbm.sport_bet_master_id = ss.sport_bet_master_id
where cts.sport_bet_master_id is not null
  and ctd.transaction_date AT TIME ZONE 'UTC -6' BETWEEN '2024-10-10 00:00:00' AND '2024-10-12 23:59:59';

Após termos os ids das apostas, podemos pegar as informações de apostas esportivas nas tabelas de sport_bets, sport_bet_masters, customers_tdt e customer_details_tdt

In [ ]:
%%sql
select
    ct.customer_id,
    sob.sport_bet_master_id as bet_master_id,
    (
        select cll.ip
        from customer_login_logs cll
        where cll.customer_id = ct.customer_id and
            cll.date_time <= sob.create_user_date
        order by cll.date_time desc
        limit 1
    ) as ip_address,
    CASE
        WHEN sob.total_play = 1 THEN '1'
        ELSE '2'
        END as bet_type,
    CASE
        WHEN sob.status = 'O' THEN '1'
        WHEN sob.status = 'W' THEN '2'
        WHEN sob.status = 'L' THEN '3'
        WHEN sob.status = 'V' THEN '3'
        WHEN sob.status = 'C' THEN '2'
        ELSE '2'
        END as bet_status,
    sob.bet_rate as fixed_odds,
    sbm.initial_played_amount as bet_amount,
    sob.return_amount as return_amount,
    CASE
        WHEN sob.status = 'C' THEN true
        ELSE false
        END as is_cashout,
    CASE
        WHEN sbm.is_system_bet = 1 THEN true
        ELSE false
        END as is_system_bet,
    sob.create_user_date AT TIME ZONE 'UTC-3' as bet_timestamp,
    sob.update_date AT TIME ZONE 'UTC-3' as update_timestamp
from sport_bets sob
         join sport_bet_masters sbm on sob.sport_bet_master_id = sbm.sport_bet_master_id
         join customers_tdt ct on sob.customer_id = ct.customer_id
         join customer_details_tdt cdt on ct.customer_id = cdt.customer_id
where sob.sport_bet_master_id IN (123);

Aqui pegamos as informações de eventos que foram apostados

In [ ]:
%%sql
select
    ss.fixture_id as event_id,
    ss.sport_type_id as sport_code,
    ss.league_name as competition,
    ss.fixture_information as event_name,
    CASE
        WHEN ss.status = 'O' THEN '4'
        ELSE '5'
    END as event_status,
    ss.fixture_start_date as event_start_date
from sport_selection ss
where
    ss.sport_bet_master_id IN (123);

E por final pegamos os mercados dos eventos de cada aposta do dia

In [ ]:
%%sql
select
    ss.sport_bet_detail_id as market_id,
    ss.fixture_id as event_id,
    ss.sport_bet_master_id as bet_master_id,
    ss.bet_type_name as market_name,
    '1' as market_type,
    ss.played_odd as fixed_odds_market,
    ss.played_amount as market_value,
    CASE
        WHEN ss.status = 'O' THEN 'PENDING'
        WHEN ss.status = 'C' THEN 'CASHOUT'
        WHEN ss.status = 'W' THEN 'WON'
        WHEN ss.status = 'L' THEN 'LOST'
        WHEN ss.status = 'V' THEN 'CANCELLED'
        ELSE 'SUSPENDED'
    END as market_result
from sport_selection ss
where
    ss.sport_bet_master_id IN (123);